# Hackathon Query Notebook

Welcome to the hackathon! This notebook is your starting point for querying the main dataset.

It's designed to connect to a **remote Tentris server** that is already running and loaded with all the data. You do not need to run or install Tentris yourself.

**Your only task:** Follow the setup steps, run the cells, and start writing your own queries!

### Step 1: Setup env

Follow the instructions in the [README.md](https://github.com/sdsc-ordes/open-pulse-quickstart/blob/feat/tentris-quickstart/src/tentris-quickstart/README.md)

### Step 2: Imports and Configuration

This cell imports the libraries and sets up our connection variables.

In [ ]:
import rdflib
import pandas as pd
import tentris 
from tentris import TentrisHTTPStore
import urllib.error
from IPython.display import display, Markdown
import os
from dotenv import load_dotenv
import requests
from pathlib import Path

load_dotenv()

#Configuration
ENDPOINT_URL = "http://128.178.219.51:7502"

if os.path.isdir(Path("/secrets")):
    display("Using Renku secrets to configure Tentris.")
    secrets_dir = Path("/secrets")
    USER = (secrets_dir / "tentris_user").read_text()
    PASS = (secrets_dir / "tentris_password").read_text()
else: 
    display("Using local env file to configure Tentris.")
    USER = os.getenv("TENTRIS_USER") 
    PASS = os.getenv("TENTRIS_PASSWORD")


if USER and PASS:
    display(Markdown("✅ Environment loaded successfully."))
else:
    display(Markdown("⚠️ Could not find any user and password for Tentris. check your secrets or .env file."))

def login(url: str, user: str, password: str) -> str:
    s = requests.Session()
    s.post(f"{url}/login", {"username": user, "password": password})
    c = s.cookies.get("tentris")
    
    return f"tentris={c}"

graph = None

'Using local env file to configure Tentris.'

✅ Environment loaded successfully.

### Step 3: Connect to the Server

This cell creates the `rdflib.Graph` object. It uses the `TentrisHTTPStore`, which is optimized to work with the Tentris server.

**Note:** This cell also runs a test query (`ASK { ?s ?p ?o }`) to make sure the server is reachable. If this cell fails, please double-check the `ENDPOINT_URL` you set in Step 2.

In [2]:
# --- Connect to Tentris Server ---

def login(url: str, user: str, password: str) -> str:
    s = requests.Session()
    resp = s.post(f"{url}/login", {"username": user, "password": password})
    resp.raise_for_status()
    cookie = s.cookies.get("tentris")
    return f"tentris={cookie}"

display(Markdown(f"🚀 Connecting to `{ENDPOINT_URL}`..."))

cookie = login(ENDPOINT_URL, USER, PASS)
display(Markdown(f"✅ Logged in as `{USER}`"))

graph = rdflib.Graph(store=TentrisHTTPStore(ENDPOINT_URL, headers={"Cookie": cookie}))

results = graph.query("SELECT * WHERE { ?s ?p ?o } LIMIT 10")
for row in results:
    print(row)



🚀 Connecting to `http://128.178.219.51:7502`...

✅ Logged in as `test`

(rdflib.term.BNode('3xfab7vzfhyw2676ijcczmhorsddr6yn'), rdflib.term.URIRef('https://open-pulse.epfl.ch/ontology#firstCommitDate'), rdflib.term.Literal('2014-06-27'))
(rdflib.term.BNode('3xfab7vzfhyw2676ijcczmhorsddr6yn'), rdflib.term.URIRef('https://open-pulse.epfl.ch/ontology#lastCommitDate'), rdflib.term.Literal('2016-03-23'))
(rdflib.term.BNode('3xfab7vzfhyw2676ijcczmhorsddr6yn'), rdflib.term.URIRef('https://open-pulse.epfl.ch/ontology#totalCommits'), rdflib.term.Literal('2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.BNode('ubc1e7pv5l7spq8imfmlikmyd4xn6w7d'), rdflib.term.URIRef('https://open-pulse.epfl.ch/ontology#email'), rdflib.term.Literal('3d101a4d3b30@me.com'))
(rdflib.term.BNode('ubc1e7pv5l7spq8imfmlikmyd4xn6w7d'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://schema.org/Person'))
(rdflib.term.BNode('ubc1e7pv5l7spq8imfmlikmyd4xn6w7d'), rdflib.term.URIRef('http://schema.org/name'), 

### Step 4: Run Example Queries

Now you're ready to go! The `graph` object is your gateway to the database.

Here are a few examples to get you started. You can (and should!) modify these and create new cells to write your own.

Use the helper function run_query() to easily run a single or multiline SPARQL query.

In [3]:
# Example 1: Count all triples in the database

def run_query(query: str, description: str = ""):
    if description:
        display(Markdown(f"**Running query:** {description}"))
    results = graph.query(query)
    df = pd.DataFrame([r.asdict() for r in results])
    display(df)

# --- Example Queries ---

run_query("SELECT (COUNT(*) AS ?totalTriples) WHERE { ?s ?p ?o }", "Counting all triples")
run_query("SELECT * WHERE { ?s ?p ?o } LIMIT 10", "Showing 10 example triples")


**Running query:** Counting all triples

,totalTriples
0,129621


**Running query:** Showing 10 example triples

,s,p,o
0,3xfab7vzfhyw2676ijcczmhorsddr6yn,https://open-pulse.epfl.ch/ontology#firstCommi...,2014-06-27
1,3xfab7vzfhyw2676ijcczmhorsddr6yn,https://open-pulse.epfl.ch/ontology#lastCommit...,2016-03-23
2,3xfab7vzfhyw2676ijcczmhorsddr6yn,https://open-pulse.epfl.ch/ontology#totalCommits,2
3,ubc1e7pv5l7spq8imfmlikmyd4xn6w7d,https://open-pulse.epfl.ch/ontology#email,3d101a4d3b30@me.com
4,ubc1e7pv5l7spq8imfmlikmyd4xn6w7d,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/Person
5,ubc1e7pv5l7spq8imfmlikmyd4xn6w7d,http://schema.org/name,Max Countryman
6,ubc1e7pv5l7spq8imfmlikmyd4xn6w7d,https://open-pulse.epfl.ch/ontology#commits,n7sot4goqqm1uf280m5zt5k6m2s34740
7,58h95tl1khfdum7thnd060fmt4x4rb65,https://open-pulse.epfl.ch/ontology#email,b21819cfebaa@hotmail.com
8,58h95tl1khfdum7thnd060fmt4x4rb65,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/Person
9,58h95tl1khfdum7thnd060fmt4x4rb65,http://schema.org/name,manolis


### Step 5: Your Turn! (Happy Hacking)

This is your canvas. Create new code cells below this one and start building your project.

**Tip:** Don't forget to add `LIMIT 10` to your queries while you are exploring, so you don't accidentally try to print a million rows!

In [5]:
run_query("""
PREFIX schema: <http://schema.org/>
PREFIX pulse: <https://open-pulse.epfl.ch/ontology#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?repoName ?disciplineIRI ?disciplineDescription
WHERE {
  ?repo a schema:SoftwareSourceCode ;
        schema:name ?repoName ;
        pulse:discipline ?disciplineIRI .
  SERVICE <https://query.wikidata.org/sparql> {
    ?disciplineIRI schema:description ?disciplineDescription .
    FILTER(LANG(?disciplineDescription) = "en")
  }
}
LIMIT 100
""", "Federated query with Wikidata")


**Running query:** Federated query with Wikidata

""


In [6]:
run_query(""" PREFIX schema: <http://schema.org/> 
    PREFIX pulse: <https://open-pulse.epfl.ch/ontology#> 
    PREFIX wd: <http://www.wikidata.org/entity/>

SELECT ?personName (COUNT(DISTINCT ?repo) AS ?repoCount)
WHERE {
  # 1. Find repositories matching the criteria
  ?repo a schema:SoftwareSourceCode ;
        schema:programmingLanguage ?language ;
        pulse:discipline wd:Q2329 . # wd:Q2329 is the IRI for Chemistry
  
  # 2. Filter for Python (case-insensitive)
#   FILTER(CONTAINS(LCASE(STR(?language)), "python"))
  
  # 3. Find the author of these repos
  ?repo schema:author ?person .
  
  # 4. Ensure the author is a Person and get their name
  ?person a schema:Person ;
          schema:name ?personName .
}
GROUP BY ?person ?personName
ORDER BY DESC(?repoCount)
LIMIT 10
""", "Top 10 authors for Python repos in Chemistry")



**Running query:** Top 10 authors for Python repos in Chemistry

""
